In [47]:
import numpy as np
import matplotlib.pyplot as plt

# All constants

R_meas = 0.04     # Nominal cell internal resistance
Ea = 40900        # Cell DISCHARGE activation energy
Rg = 8.314        # Ideal gas constant
T_ref= 298.15     # in K
F     = 96485.33   # C/mol Faraday's Constant
n     = 1          # electrons per reaction (for LiFePO4)
E0    = 3.3       # V   — cell’s nominal standard potential
dS    = -0.0001    # V/K — entropy term
alpha = 1.0

class OCVSOCKalmanFilter:
    def __init__(self, process_noise=1e-8, measurement_noise=0.0036):
        self.ocv = 0.0
        self.P = 0.01
        self.A = 1.0
        self.H = 1.0
        self.Q = process_noise
        self.R = measurement_noise
        self.smoothed_ocv = []
        self.uncertainties = []

    def smooth_curve(self, soc_points, ocv_measurements):
        self.smoothed_ocv = []
        self.uncertainties = []
        self.ocv = ocv_measurements[0]
        self.P = self.R

        for i, (soc, ocv_measured) in enumerate(zip(soc_points, ocv_measurements)):
            self.predict()
            self.update(ocv_measured)
            self.smoothed_ocv.append(self.ocv)
            self.uncertainties.append(np.sqrt(self.P))

        return np.array(self.smoothed_ocv), np.array(self.uncertainties)

    def predict(self):
        self.ocv = self.A * self.ocv
        self.P = self.A * self.P * self.A + self.Q

    def update(self, ocv_measured):
        K = self.P * self.H / (self.H * self.P * self.H + self.R)
        innovation = ocv_measured - self.H * self.ocv
        self.ocv = self.ocv + K * innovation

        self.P = (1 - K * self.H) * self.P

    def generate_noisy_lfp_curve(self, T_C, noise_std=0.02):
      T_K_local = T_C + 273.15
      soc = np.linspace(0, 1, 101)
      soc_clip = np.clip(soc, 1e-6, 1 - 1e-6)
      ocv_true = (
        E0
        + (R_g * T_K_local) / (n * F) * np.log(soc_clip / (1 - soc_clip))
        + dS * (T_K_local - T_ref))
      rng = np.random.default_rng(42)
      ocv_noisy = ocv_true + rng.normal(0, noise_std, size=soc.shape)
      return soc, ocv_true, ocv_noisy

    def test_ocv_smoothing(self, T_C):
        print("OCV/SOC Curve Smoothing Test")
        soc, ocv_true, ocv_noisy = self.generate_noisy_lfp_curve(T_C)
        smoothers = {
            'Conservative': OCVSOCKalmanFilter(process_noise=1e-6, measurement_noise=1e-4),
            'Moderate': OCVSOCKalmanFilter(process_noise=1e-5, measurement_noise=1e-3),
            'Aggressive': OCVSOCKalmanFilter(process_noise=1e-4, measurement_noise=1e-2)
        }
        return soc, ocv_true, ocv_noisy, smoothers

    def demonstrate_parameter_tuning(self, soc, ocv_true, ocv_noisy):
        print("\n=== Parameter Tuning Guide ===")
        print("Q (process noise) controls smoothness: 1e-6 → very smooth; 1e-4 → follows noise more closely.")
        print("R (measurement noise) reflects data trust: 1e-4 → trust measurements; 1e-2 → assume noisy.")

        q_values = [1e-7, 1e-6, 1e-5, 1e-4]
        r_values = [1e-4, 1e-3, 1e-2]

        print("Parameter Sensitivity (RMSE in mV)")
        print("Q\\R     ", end="")
        for r in r_values:
            print(f"{r:8.0e}", end="")
        print()

        for q in q_values:
            print(f"{q:8.0e} ", end="")
            for r in r_values:
                smoother = OCVSOCKalmanFilter(process_noise=q, measurement_noise=r)
                smoothed, _ = smoother.smooth_curve(soc, ocv_noisy)
                rsme = np.sqrt(np.mean((smoothed - ocv_true)**2)) * 1000
                print(f"{rsme:8.2f}", end="")
            print()


if __name__ == "__main__":
    # ——— USER PROMPTS ———
    target_soc   = float(input("Enter target SOC (0 – 1): "))
    T_C          = float(input("Enter temperature (°C): "))
    target_weeks = float(input("Enter time since shipment (weeks): "))

    print(f"\nRunning SOH estimation for SOC={target_soc:.2f}, T={T_C:.1f}°C, t={target_weeks:.1f} weeks\n")

kalman_filter_instance = OCVSOCKalmanFilter()
soc_arr, ocv_true, ocv_noisy, smoothers = kalman_filter_instance.test_ocv_smoothing(T_C)
kalman_filter_instance.demonstrate_parameter_tuning(soc_arr, ocv_true, ocv_noisy)

def plot_ocv_smoothing(soc_arr, ocv_true, ocv_noisy, smoothers):

    plt.figure(figsize=(15, 10))
    plt.subplot(2, 2, 1)
    plt.plot(soc_arr * 100, ocv_true, 'g-', linewidth=2, label='True OCV')
    plt.plot(soc_arr * 100, ocv_noisy, 'r.', alpha=0.5, markersize=3, label='Noisy')
    for (name, smoother), color in zip(smoothers.items(),
                                       ['lightblue','lightgreen','hotpink']):
        sm, _ = smoother.smooth_curve(soc_arr, ocv_noisy)
        plt.plot(soc_arr * 100, sm, color=color, linewidth=2, label=name)
    plt.title('OCV/SOC Curve Smoothing Test'), plt.xlabel('SOC (%)'), plt.ylabel('OCV (V)')
    plt.legend(), plt.grid(alpha=0.3)

    plt.subplot(2, 2, 2)
    mask = (soc_arr >= 0.2) & (soc_arr <= 0.8)
    plt.plot(soc_arr[mask] * 100, ocv_true[mask], 'g-', linewidth=2)
    plt.plot(soc_arr[mask] * 100, ocv_noisy[mask], 'r.', alpha=0.5, markersize=3)
    for (name, smoother), color in zip(smoothers.items(),
                                       ['lightblue','lightgreen','hotpink']):
        sm, _ = smoother.smooth_curve(soc_arr, ocv_noisy)
        plt.plot(soc_arr[mask] * 100, sm[mask], color=color, linewidth=2, label=name)
    plt.title('Plateau Region (20–80%)'), plt.xlabel('SOC (%)'), plt.ylabel('OCV (V)')
    plt.legend(), plt.grid(alpha=0.3)

    plt.subplot(2, 2, 3)
    for (name, smoother), color in zip(smoothers.items(),
                                       ['lightblue','lightgreen','hotpink']):
        _, unc = smoother.smooth_curve(soc_arr, ocv_noisy)
        plt.plot(soc_arr * 100, unc * 1000, color=color, linewidth=2, label=name)
    plt.title('Uncertainty (mV)'), plt.xlabel('SOC (%)'), plt.ylabel('σ (mV)')
    plt.legend(), plt.grid(alpha=0.3)

    plt.subplot(2, 2, 4)
    for (name, smoother), color in zip(smoothers.items(),
                                       ['lightblue','lightgreen','hotpink']):
        sm, _ = smoother.smooth_curve(soc_arr, ocv_noisy)
        err = abs(sm - ocv_true) * 1000
        plt.plot(soc_arr * 100, err, color=color, linewidth=2, label=name)
    raw_err = abs(ocv_noisy - ocv_true) * 1000
    plt.plot(soc_arr * 100, raw_err, 'r--', alpha=0.7, label='Raw Error')
    plt.title('Error vs. Raw Data (mV)'), plt.xlabel('SOC (%)'), plt.ylabel('Error (mV)')
    plt.legend(), plt.grid(alpha=0.3)
    plt.tight_layout()

_show = input("Show SOH Smoothing with Kalman Filter diagnostic plots ? (y/n): ").strip().lower()
if _show in ('y', 'yes'):
  plot_ocv_smoothing(soc_arr, ocv_true, ocv_noisy, smoothers)
  plt.show()
  print("\nPerformance Metrics:")
  print("\nMost confident OCV estimates")
  target_idx = np.argmin(np.abs(soc_arr - target_soc))
  for name, smoother in smoothers.items():
    smoothed, uncertainties = smoother.smooth_curve(soc_arr, ocv_noisy)
    ocv_at_target = smoothed[target_idx]   # in V
    unc_at_target = uncertainties[target_idx] * 1000  # in mV
    print(f"{name}: SOC\u2248{soc_arr[target_idx]:.4f} \u2192 Smoothed OCV = {ocv_at_target:.4f} V \u00B1{unc_at_target:.1f} mV")

else:
    print("Skipping diagnostic plots.")

    print("\nPerformance Metrics:")
    print("\nMost confident OCV estimates")
    target_idx = np.argmin(np.abs(soc_arr - target_soc))

    for name, smoother in smoothers.items():
        smoothed, uncertainties = smoother.smooth_curve(soc_arr, ocv_noisy)
        ocv_at_target = smoothed[target_idx]   # in V
        unc_at_target = uncertainties[target_idx] * 1000  # in mV

        print(f"{name}: SOC\u2248{soc_arr[target_idx]:.4f} \u2192 Smoothed OCV = {ocv_at_target:.4f} V \u00B1{unc_at_target:.1f} mV")

# ----- Synthetic Data Generation (Weeks Since Shipped) -----

# Time Axis (weeks since start)
N = 1000
dt = 1.0  # 1 week per step
times = np.arange(N) * dt  # in weeks

# Temperature and SOC profiles
T_K = np.full(N, T_C + 273.15)
SOC = (np.full_like(times, target_soc))*100


def R_arr(T_K):
  R_arr_val = R_meas* np.exp(Ea/Rg*((1/T_K)-(1/T_ref)))
  return R_arr_val


# CALCE calendar‐aging model expects time in months
def Ri_increase_months(t_months, T_K, SOC):
    T_C_local = T_K - 273.15
    A = (0.3719 * np.exp(0.05168 * T_C_local) * np.exp(0.005033 * SOC)
        - 0.287 * np.exp(0.05168 * T_C_local)
        + 2.618 * np.exp(0.005033 * SOC)
        - 2.021
    )
    B = -0.1104 * np.exp(0.01399 * SOC) + 0.9721

    R0 = (0.3719 * np.exp(0.05168 * 25) * np.exp(0.005033 * 50)       # R0 is the model's baseline resistance
         - 0.287 * np.exp(0.05168 * 25)
         + 2.618 * np.exp(0.005033 * 50)
         - 2.021) * (1 ** (-0.1104 * np.exp(0.01399 * 50) + 0.9721)) / 1e3
    return (A * (t_months ** B))/1e3 - R0

# Total internal resistance model, input t in weeks
def R_int_model(t_weeks, T_K, SOC):
    arr_term = R_arr(T_K)
    # convert weeks to months for CALCE model
    t_months = t_weeks/4.34524
    inc_term = Ri_increase_months(t_months, T_K, SOC)
    return arr_term + inc_term

# Convert resistance to SOH
def soh_from_R(R_int):
    return np.clip(np.exp(-alpha * (R_int - R_meas)), 0.0, 1.0)   # Use an exponential model

soh_ref = soh_from_R(R_int_model(times, T_K, SOC))

# Generate synthetic SOH data with noise
noise_std = 0.02
soh_ref = soh_from_R(R_int_model(times, T_K, SOC))
soh_noisy = soh_ref + np.random.normal(0, noise_std, size=N)
soh_noisy = np.clip(soh_noisy, 0.0, 1.0)  # Ensure SOH values are within [0, 1]

# ---- Stage 1: Basic 1D Kalman Smoothing ----
class SOHKalmanFilter1:
    def __init__(self, Q=1e-6, R=noise_std**2):
        self.Q = Q
        self.R = R
    def smooth(self, z):
        x = z[0]
        P = self.R
        x_est = np.zeros_like(z)
        for k, zk in enumerate(z):
            P += self.Q
            K = P / (P + self.R)
            x += K * (zk - x)
            P *= (1 - K)
            x_est[k] = x
        return x_est

kf1 = SOHKalmanFilter1()
soh_stage1 = kf1.smooth(soh_noisy)

# ---- Stage 2: Adaptive Kalman with Hybrid Q and R(T) ----
class SOHKalmanFilter2:
    def __init__(self, R_func, alpha=1.0, w=0.3, Q_min=1e-10, Q_R_min=1e-14, adapt_gain_R=0.1, adapt_gain_Q=0.1):
        self.R_func = R_func
        self.alpha = alpha
        self.w = w
        self.Q_min = Q_min
        self.Q_R_min = Q_R_min
        self.adapt_gain_R = adapt_gain_R
        self.adapt_gain_Q = adapt_gain_Q

    def Q_drift(self, t, dt, T, SOC):
        R1 = R_int_model(t, T, SOC)
        R2 = R_int_model(t + dt, T, SOC)
        soh1 = soh_from_R(R1)
        soh2 = soh_from_R(R2)
        return max((soh2 - soh1)**2, self.Q_min)

    def Q_jacobian(self, t, dt, T, SOC):
        R1 = R_int_model(t, T, SOC)
        R2 = R_int_model(t + dt, T, SOC)
        dR = R2 - R1
        Q_R = max(dR**2, self.Q_R_min)
        J = abs(-self.alpha * soh_from_R(R1)/R1 )  # Corrected Jacobian calculation
        return (J**2) * Q_R

    def run(self, z, times, Ts, SOCs, dt):
        N = len(z)
        x_est = np.zeros(N)
        unc   = np.zeros(N)
        x = z[0]
        P = self.R_func(Ts[0])
        for k in range(N):
            t = times[k]
            T = Ts[k]
            soc_local = SOCs[k] # Use a local variable for SOC
            z_k = z[k]
            q1 = self.Q_drift(t, dt, T, soc_local) # Use local SOC
            q2 = self.Q_jacobian(t, dt, T, soc_local) # Use local SOC
            Qk = self.w * q2 + (1 - self.w) * q1
            Rk = self.R_func(T)
            P += Qk

            x_pred = x
            innovation = z[k] - x_pred

            Rk = Rk + self.adapt_gain_R * (innovation**2 - Rk)
            Qk = max(self.Q_min,
                     Qk + self.adapt_gain_Q * (innovation**2 - Qk))   # Adaptive Q and R Variances


            K = P / (P + Rk)
            x += K * (z_k - x)
            x=max(0.0,min(1.0,x))
            P *= (1 - K)
            x_est[k] = x
            unc[k]   = np.sqrt(P)
        return x_est, unc

# Measurement-noise function (Arrhenius-based)
def R_func(T_K):
    base_var=(0.01)**2
    arr = 10**(Ea/(2.303*Rg*T_K))
    arr_norm = (10**(Ea/(2.303*Rg*298.15)))
    return base_var*(arr/arr_norm)

# Initialize and run the adaptive Kalman filter
kf2 = SOHKalmanFilter2(R_func=R_func, w=0.7)
soh_stage2, unc_stage2 = kf2.run(soh_stage1, times, T_K, SOC, dt)

_show = input("Show SOH Smoothing with Kalman Filter diagnostic plots ? (y/n): ").strip().lower()
if _show in ('y', 'yes'):
  plt.figure(figsize=(12, 6))
  plt.plot(times, soh_ref, 'k-', linewidth=2, label='Reference SOH')
  plt.plot(times, soh_noisy, 'r.', alpha=0.5, markersize=3, label='Noisy SOH')
  plt.plot(times, soh_stage1, 'b-', linewidth=2, label='Kalman Stage 1')
  plt.plot(times, soh_stage2, 'g-', linewidth=2, label='Adaptive Kalman Stage 2')
  plt.xlabel('Time (Weeks)')
  plt.ylabel('SOH')
  plt.title('SOH Smoothing with Kalman Filters')
  plt.legend()
  plt.grid(True, alpha=0.3)
  plt.show()



  plt.figure(figsize=(12, 6))
  plt.plot(times, soh_ref, 'k-', label='Reference SOH')
  plt.plot(times, soh_noisy, 'r.', alpha=0.5, label='Noisy SOH')
  plt.plot(times, soh_stage1, 'b-', label='Stage 1 Smoothed')
  plt.plot(times, soh_stage2, 'g-', label='Stage 2 Estimate')
  plt.fill_between(times,
                 soh_stage2 - unc_stage2,
                 soh_stage2 + unc_stage2,
                 color='green', alpha=0.2,
                 label='±1σ Uncertainty')
  plt.xlabel('Time (weeks)')
  plt.ylabel('SOH')
  plt.title('SOH Estimation via Dual-Stage Kalman Filter')
  plt.legend()
  plt.grid(alpha=0.3)
  plt.show()

else:
    print("Skipping diagnostic plots.")

  # ---- Query best-guess at arbitrary week ----
t_star = target_weeks
soh_q  = np.interp(t_star, times, soh_stage2)
unc_q  = np.interp(t_star, times, unc_stage2)

print("---------------------------------------------------------------------------------------------------")
print(f"Best State of Health Estimate at week {t_star:.1f}: {soh_q*100:.2f}% \u00B1 {unc_q*100:.2f}%")
print(f"State of Charge: {SOC[0]:.4f}")
print(f"Temperature used: {T_K[0] - 273.15:.4f} \u00B0C")

Enter target SOC (0 – 1): .2
Enter temperature (°C): 25
Enter time since shipment (weeks): 1

Running SOH estimation for SOC=0.20, T=25.0°C, t=1.0 weeks

OCV/SOC Curve Smoothing Test

=== Parameter Tuning Guide ===
Q (process noise) controls smoothness: 1e-6 → very smooth; 1e-4 → follows noise more closely.
R (measurement noise) reflects data trust: 1e-4 → trust measurements; 1e-2 → assume noisy.
Parameter Sensitivity (RMSE in mV)
Q\R        1e-04   1e-03   1e-02
   1e-07    60.69   70.51   72.30
   1e-06    45.90   60.69   70.51
   1e-05    32.43   45.90   60.69
   1e-04    18.60   32.43   45.90
Show SOH Smoothing with Kalman Filter diagnostic plots ? (y/n): n
Skipping diagnostic plots.

Performance Metrics:

Most confident OCV estimates
Conservative: SOC≈0.2000 → Smoothed OCV = 3.2340 V ±3.1 mV
Moderate: SOC≈0.2000 → Smoothed OCV = 3.2340 V ±9.9 mV
Aggressive: SOC≈0.2000 → Smoothed OCV = 3.2340 V ±31.2 mV
Show SOH Smoothing with Kalman Filter diagnostic plots ? (y/n): n
Skipping diag